# Transferência de aprendizado
Desafio: contruir uma arquitetura capaz de classificar imagens entre Muffin ou Chihuahua com base no dataset https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification?select=test utilizando a transferência de aprendizado de duas redes já existentes a VGG-16 e a ResNet50. 

A VGG-16 é uma rede neural de aprendizado profundo criada em 2014 formada de uma pilha de camadas de Convolução, MaxPooling e conectadas por camadas Dense.

Já a rede ResNet50 foi criada em 2015 com o objetivo de lidar com o problema de "degradação de desempenho" onde a rede profunda começa a perde a eficácia ao adicionar camadas adicionais, para isso foi adotada uma abordagem onde camadas adicionai são inseridas com a sima dos resultados da camada anterior e da camada atual.

O Código abaixo refere-se a preparação do ambiente 

In [ ]:
!pip install keras

In [ ]:
from keras.applications import VGG16, ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping 

Importando as duas redes VGG-16 e ResNet50

In [ ]:
vgg16_model = VGG16(
    weights='imagenet', # tipo de treinamento que foi realizado nessa rede
    input_shape = (224,224,3) # dimensões da imagem de entrada
)
resnet50_model = ResNet50(
    weights='imagenet', # tipo de treinamento que foi realizado nessa rede
    input_shape = (224,224,3) # dimensões da imagem de entrada
)

Como vamos realizar uma transferência de aprendizado é importante “congelar” os pesos das camadas que já foram ajustados. Essa estratégia também previne o overfitting e torna o treinamento mais rápido.

In [ ]:
# Percorre todas as camadas da VGG16 e seta o "trainable" como False
for l in vgg16_model.layers:
    l.trainable = False

for l in resnet50_model.layers:
    l.trainable = False

O código a seguir é responsável por concatenar as redes VGG16 e ResNet50. 
Para isso será criado uma camada de input, concatenado as redes VGG15 e ResNet50 e posteriormente duas camadas densas.

In [ ]:
# camada de entrada
input_tensor = Input(shape=(224,224,3))

vgg16_outputs = vgg16_model(input_tensor)

resnet50_outputs = resnet50_model(input_tensor)

# Concatenando
merged_outputs = Concatenate()([vgg16_outputs, resnet50_outputs])

# Criando uma camada densa 
modelo = Dense(128, activation = 'relu')(merged_outputs)
               #.        #               # A saída de merge é a entrada dessa camada
               #.        # Impedir números negativos 
               #. quantidade de neurôneos  
modelo = Dense(2, activation = 'softmax')(modelo)
               #         #                #A saída do modelo é a entrada dessa camada 
               #         # converter as saídas em probabilidade,
               # Quantidade de neurôneos 

# Cria o modelo final 
modelo_final = Model(inputs=input_tensor, outputs=modelo)

Visualizar a arquitetura criada

In [ ]:
modelo_final.summary()

O Modelo está estruturado, porém para dar continuidade é necessário compilar

In [ ]:
modelo_final.compile(
    optimizer='adam', # Otimizador de pesos 
    loss='categorical_crossentropy', #Função de perda para avaliação de desempenho 
    metrics=['accuracy'] # definir a métrica para avaliar o desempenho
)

As linhas a seguir são responsáveis por acessar os arquivos de estudo do problema

In [ ]:
diretorio_treino = "/content/drive/.../muffin-chihuahua/train"
diretorio_validacao = "/content/drive/.../muffin-chihuahua/test"

# Analisando as imagens, é notado a necessidade de normalizar 
# com relação a quantidade de Bits.identifiquei que elas possuem dimensões diferentes
# O código abaixo prepara uma variável que receberá as imagens, 
# ele está pre configurado para torma as imagens no padrão 8 bits
dados_treino = ImageDataGenerator(
    rescale=1./255
)

dados_validacao = ImageDataGenerator(
    rescale=1./255
)

# Carrega as imagens da pasta e prepara para serem usadas como entrada em uma rede
dados_treino = dados_treino.flow_from_directory(
    diretorio_treino, # caminho dos arquivos 
    target_size=(224,224), # redimencionando as imagens para 224X224
    class_mode= "categorical", # define o tipo de saida categorica
    batch_size= 32 # numero de amostras usadas para cada interação de treinamento
)

dados_validacao = dados_validacao.flow_from_directory(
    diretorio_validacao, 
    target_size=(224,224),
    class_mode= "categorical", 
    batch_size= 32 
)

Chegou o momento de treinar a Rede.

In [ ]:
modelo_treinado = modelo_final.fit( 
    dados_treino, 
    steps_per_epoch = 32,
    epochs = 20,
    validation_data = dados_validacao,
    callbacks=[EarlyStopping(monitor = 'val_accuracy', patience = 2)],
    verbose = 1
)

Aqui explico a função de cada linha 

**dados_treino** - corresponde ao nosso diretório com as imagens utilizadas para o treino da rede.

**steps_per_epoch** - define o número de passos a serem feitos em cada época

**epochs** - número de epocas 

**validation_data** - dados usados para a validação 

**callbacks**- o "EarlyStopping" monitora a evolução da rede pelo atributo "val_accuracy", caso não hava evoluçã em duas épocas a rede encerra o treinamento 

**verbose** -  define que o processo de treinamento deverá aparece no console 

In [ ]:
#como estou fazendo vários testes vou salvar em disco o modelo
modelo_final.save('/content/drive/..../muffin-chihuahua/modelo_final.h5')
# Gerou um arquivo com pouco mais de 600MB 

# código para carregar um modelo em disco 
# from keras.models import load_model 
# modelo_final_carregado = load_model('/content/drive/.../muffin-chihuahua/modelo_final.h5')


Agora chegou a hora dos testes

No meu Drive, coloquei algumas fotos de cachorros e bolinhos para testar 

In [ ]:
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np 

# imagem que usarei para teste 
image_cachorro_teste = "/content/drive/.../muffin-chihuahua/chihuahua_teste1.jpeg"

# redimensiona a imagem para entrada da rede neural 
img = image.load_img(image_cachorro_teste, target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# solicita para ele predizer a imagem enviada 
preds = modelo_final.predict(x)

In [ ]:
print("Probabilidade ")
for classe, indice in dados_validacao.class_indices.items():
  print("{} : {:.0%}".format(classe, preds[0][indice]))
  